In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/WEB_MINING_PROJECT/BERT'
except ImportError as e:
    pass

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1EZ3t7HMzSmWcvRl80eFHiMW9UP3zMZxI/WEB_MINING_PROJECT/BERT


In [3]:
!pip install --upgrade tensorflow
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [4]:
!pip show transformers

Name: transformers
Version: 4.40.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
import numpy as np

In [6]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
df = pd.read_csv('downsampled_dataset_10k.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   marketplace        9999 non-null   object
 1   customer_id        9999 non-null   int64 
 2   review_id          9999 non-null   object
 3   product_id         9999 non-null   object
 4   product_parent     9999 non-null   int64 
 5   product_title      9999 non-null   object
 6   product_category   9999 non-null   object
 7   star_rating        9999 non-null   int64 
 8   helpful_votes      9999 non-null   int64 
 9   total_votes        9999 non-null   int64 
 10  vine               9999 non-null   object
 11  verified_purchase  9999 non-null   object
 12  review_headline    9999 non-null   object
 13  review_body        9999 non-null   object
 14  review_date        9999 non-null   object
 15  sentiment          9999 non-null   object
dtypes: int64(5), object(11)
memory usage: 1.2+

In [8]:
df['sentiment'].value_counts()

sentiment
Positive    7635
Negative    1680
Neutral      684
Name: count, dtype: int64

In [9]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,6108596,RN8YG8Q0AS227,B00P8LFJ3Y,406643994,Lava HD-8000 OmniPro,Electronics,5,0,0,N,Y,Five Stars,Great tv signal very good buy.I like it,2015-08-21,Positive
1,US,37870958,R1U4X7M4TMY84A,B00WBS32K4,527761468,PonoMusic Pono Portable Music Player,Electronics,5,1,2,N,Y,Five Stars,Good sound. Fine Material. Simply perfect!,2015-08-29,Positive
2,US,38204831,R291XVGVS56XGR,B00XVT4DLO,80192107,DIVOIX® DV110 In-Ear Headphone Earbuds Lightwe...,Electronics,5,0,0,N,Y,Five Stars,looks good as shown lots of base. BTW fast shi...,2015-08-22,Positive
3,US,26611731,R2Y9PUSXVAMOUU,B00N1KWERI,97589125,"Eclipse 8GB 2.8"" Supra Fit Bluetooth MP3 with ...",Electronics,5,0,0,N,Y,Love it!,This player is totally awesome! I has all the ...,2015-08-18,Positive
4,US,47611641,R1O2U9QMV39FK2,B004LTEUDO,533949740,Mediabridge 3.5mm Male To Male Stereo Audio Ca...,Electronics,5,0,0,N,Y,High Quality Cable,Very quick delivery and high quality. Sound is...,2015-08-20,Positive


In [10]:
dataset_filtered = df[["review_body", "sentiment"]]

In [11]:
dataset_filtered.head()

,review_body,sentiment
0,Great tv signal very good buy.I like it,Positive
1,Good sound. Fine Material. Simply perfect!,Positive
2,looks good as shown lots of base. BTW fast shi...,Positive
3,This player is totally awesome! I has all the ...,Positive
4,Very quick delivery and high quality. Sound is...,Positive


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
import numpy as np

# Step 2: Split the data
train_data, test_data = train_test_split(dataset_filtered, test_size=0.2, stratify=dataset_filtered['sentiment'], random_state=42)


#Step 3: Oversampling
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(np.array(train_data['review_body']).reshape(-1, 1), train_data['sentiment'])

train_data_resampled = pd.DataFrame({'review_body': X_train_resampled.flatten(), 'sentiment': y_train_resampled})




In [14]:
# Step 4: Tokenization

# Specify the pre-trained model name.
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased' # uncased means this tokenizer will also lower-case automatically

# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


train_data_resampled['review_body'] = train_data_resampled['review_body'].astype(str)
texts = list(train_data_resampled['review_body'])

test_data['review_body'] = test_data['review_body'].astype(str)
texts1 = list(test_data['review_body'])

train_encodings = tokenizer(texts, truncation=True, padding=True)
test_encodings = tokenizer(texts1, truncation=True, padding=True)

train_labels = torch.tensor(train_data_resampled['sentiment'].map({'Positive': 2, 'Neutral': 1, 'Negative': 0}).tolist())
test_labels = torch.tensor(test_data['sentiment'].map({'Positive': 2, 'Neutral': 1, 'Negative': 0}).tolist())

# Step 5: Model Architecture
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Step 6: Training
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.train()
EPOCHS = 3
for epoch in range(3):  # example, you can adjust the number of epochs
    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Step 7: Evaluation
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             test_labels)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

from sklearn.metrics import classification_report

model.eval()
all_predictions = []
all_labels = []

for batch in test_loader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    _, predicted = torch.max(logits, 1)
    all_predictions.extend(predicted.tolist())
    all_labels.extend(labels.tolist())

# Compute classification report
report = classification_report(all_labels, all_predictions)

print("Classification Report:")
print(report)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.65      0.73       336
           1       0.28      0.44      0.34       137
           2       0.94      0.94      0.94      1527

    accuracy                           0.85      2000
   macro avg       0.69      0.67      0.67      2000
weighted avg       0.88      0.85      0.86      2000



In [15]:
predicted_sentiments = all_predictions
original_sentiments = all_labels

In [17]:
test_data["generated_sentiment"] = all_predictions

In [19]:
test_data["numeric_sentiment"] = all_labels

In [20]:
test_data.head()

,review_body,sentiment,generated_sentiment,numeric_sentiment
52,Works perfectly,Positive,2,2
8065,not comfortable to me,Neutral,0,1
9262,Bought these again after losing the last pair ...,Positive,2,2
8895,"A fine product. Stable and secure, and it's l...",Positive,2,2
482,Do not use in vaporizer equipment ever !!!!!,Negative,0,0


In [22]:
train_data_resampled.shape

(18324, 2)

In [23]:
# Store the DataFrame into a CSV file
test_data.to_csv('test_data_BERT.csv', index=False)